<a href="https://colab.research.google.com/github/faryzafarizal/UMHackathon-Wonder-Girls/blob/main/UMhackdata_fetch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CryptoQuant


In [ ]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
pip install cybotrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 60.2 MB/s eta 0:00:00


In [ ]:
!pip install load_dotenv

In [ ]:
import cybotrade
import asyncio
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from cybotrade.strategy import Strategy
from cybotrade.models import OrderSide, Exchange, RuntimeConfig, RuntimeMode
from datetime import datetime, timezone
from cybotrade.permutation import Permutation
import cybotrade_datasource
import nest_asyncio
import os
from dotenv import load_dotenv
 # Add this import at the top

In [ ]:
# Load environment variables
load_dotenv()

# Set the API key (preferably from .env file)
API_KEY = "URPhT1mp48dLh15EjeoEZf0hTTYqpkoWQ60ZsxriEdtTJi6I"
os.environ["API_KEY"] = API_KEY

# Verify the API key
print(f"API Key set: {API_KEY[:8]}...")  # Print first 8 chars for verification

API Key set: URPhT1mp...


In [21]:
# Define data sources configuration
data_sources = {
    'exchange_netflow': {
        'topic': 'cryptoquant|btc/exchange-flows/netflow',
        'params': {
            'window': 'hour',
            'exchange': 'all_exchange'
        }
    },
    'miner_to_exchange': {
        'topic': 'cryptoquant|btc/inter-entity-flows/miner-to-exchange',
        'params': {
            'window': 'hour',
            'from_miner': 'all_miner',
            'to_exchange': 'all_exchange'
        }
    },
    'exchange_reserve': {
        'topic': 'cryptoquant|btc/exchange-flows/reserve',
        'params': {
            'window': 'hour',
            'exchange': 'all_exchange'
        }
    },
    'price_ohlcv': {
        'topic': 'cryptoquant|btc/market-data/price-ohlcv',
        'params': {
            'window': 'hour',
            'market': 'spot',
            'exchange': 'all_exchange',
            'symbol': 'btc_usd'
        }
    }
}

async def fetch_data(api_key, topic, params, start_time, end_time):
    """Helper function to fetch data from CyboTrade"""
    full_topic = f"{topic}?{'&'.join([f'{k}={v}' for k,v in params.items()])}"
    try:
        data = await cybotrade_datasource.query_paginated(
            api_key=api_key,
            topic=full_topic,
            start_time=start_time,
            end_time=end_time
        )
        return pd.DataFrame(data) if isinstance(data, list) else None
    except Exception as e:
        print(f"Error fetching {topic}: {str(e)}")
        return None

async def main():
    try:
        if not API_KEY:
            raise ValueError("API key is not set")

        # Define time range
        start_time = datetime(year=2023, month=1, day=1, tzinfo=timezone.utc)
        end_time = datetime(year=2024, month=1, day=1, tzinfo=timezone.utc)

        # Dictionary to store all fetched dataframes
        dfs = {}

        # Fetch all data sources
        for source_name, config in data_sources.items():
            print(f"Fetching {source_name}...")
            # Skip mvrv since it doesn't support hourly data
            if source_name == 'mvrv':
                continue

            df = await fetch_data(
                API_KEY,
                config['topic'],
                config['params'],
                start_time,
                end_time
            )
            if df is not None:
                dfs[source_name] = df
                print(f"Successfully fetched {source_name} data: {len(df)} rows")
                display(df.head())
            print("-" * 100)

        return dfs  # Return the dictionary of dataframes

    except Exception as e:
        print(f"Error details: {str(e)}")
        return None

# Run the async function and store the result
nest_asyncio.apply()
dfs = await main()

Fetching exchange_netflow...
Successfully fetched exchange_netflow data: 8760 rows


,start_time,datetime,netflow_total
0,1672531200000,2023-01-01 00:00:00,309.663736
1,1672534800000,2023-01-01 01:00:00,392.516794
2,1672538400000,2023-01-01 02:00:00,469.784248
3,1672542000000,2023-01-01 03:00:00,207.986097
4,1672545600000,2023-01-01 04:00:00,-8.753805


----------------------------------------------------------------------------------------------------
Fetching miner_to_exchange...
Successfully fetched miner_to_exchange data: 8760 rows


,start_time,datetime,flow_mean,flow_total,transactions_count_flow
0,1672531200000,2023-01-01 00:00:00,0.110453,0.331358,3
1,1672534800000,2023-01-01 01:00:00,0.056179,0.730331,13
2,1672538400000,2023-01-01 02:00:00,3.873039,298.224006,77
3,1672542000000,2023-01-01 03:00:00,0.889167,20.450841,23
4,1672545600000,2023-01-01 04:00:00,0.157032,12.091448,77


----------------------------------------------------------------------------------------------------
Fetching exchange_reserve...
Successfully fetched exchange_reserve data: 8760 rows


,start_time,datetime,reserve,reserve_usd
0,1672531200000,2023-01-01 00:00:00,2.970859e+06,5.459904e+10
1,1672534800000,2023-01-01 01:00:00,2.971252e+06,5.464014e+10
2,1672538400000,2023-01-01 02:00:00,2.971722e+06,5.468251e+10
3,1672542000000,2023-01-01 03:00:00,2.971930e+06,5.460409e+10
4,1672545600000,2023-01-01 04:00:00,2.971921e+06,5.461159e+10


----------------------------------------------------------------------------------------------------
Fetching price_ohlcv...
Successfully fetched price_ohlcv data: 8760 rows


,start_time,close,datetime,high,low,open,volume
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297


----------------------------------------------------------------------------------------------------


In [23]:
for key, df in dfs.items():
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], utc=True)
        df.set_index('time', inplace=True)
    elif 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
        df.set_index('timestamp', inplace=True)
    dfs[key] = df.sort_index()


In [25]:
dfs.keys()


dict_keys(['exchange_netflow', 'miner_to_exchange', 'exchange_reserve', 'price_ohlcv'])

In [26]:
display(dfs['price_ohlcv'].head())

,start_time,close,datetime,high,low,open,volume
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297
